# AI based Natural disaster analysis

### Importing Neccessary Libraries

In [9]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
tensorflow.__version__

'2.8.0'

In [ ]:
tensorflow.keras.__version__

'2.8.0'

### Image Data Augmentation

In [ ]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data augmentation

In [ ]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'C:\Users\hp\Desktop\IBM\dataset\test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'C:\Users\hp\Desktop\IBM\dataset\test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 198 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [ ]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [ ]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [ ]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 64, 1: 29, 2: 61, 3: 44})

### Creating the model

In [ ]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2



In [ ]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        9

### Compiling the model

In [ ]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [ ]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/20


C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


40/40 [==============================] - 17s 440ms/step - loss: 1.3599 - accuracy: 0.3434 - val_loss: 1.3235 - val_accuracy: 0.3081
Epoch 2/20
40/40 [==============================] - 17s 421ms/step - loss: 1.2764 - accuracy: 0.3485 - val_loss: 1.3702 - val_accuracy: 0.3384
Epoch 3/20
40/40 [==============================] - 18s 448ms/step - loss: 1.0652 - accuracy: 0.5808 - val_loss: 0.7824 - val_accuracy: 0.6970
Epoch 4/20
40/40 [==============================] - 20s 514ms/step - loss: 0.9344 - accuracy: 0.6566 - val_loss: 0.7065 - val_accuracy: 0.7576
Epoch 5/20
40/40 [==============================] - 19s 468ms/step - loss: 0.7658 - accuracy: 0.6768 - val_loss: 1.0319 - val_accuracy: 0.6515
Epoch 6/20
40/40 [==============================] - 18s 447ms/step - loss: 0.7263 - accuracy: 0.7222 - val_loss: 0.6478 - val_accuracy: 0.7727
Epoch 7/20
40/40 [==============================] - 16s 397ms/step - loss: 0.6744 - accuracy: 0.7525 - val_loss: 0.6544 - val_accuracy: 0.7677
Epoch 8/20

### Saving our model

In [ ]:
# Save the model
classifier.save('disaster.h5')

In [ ]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [17]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img,img_to_array
model = load_model("/content/disaster.h5") #loading the model for testing

In [18]:
img = load_img(r"/content/e2.jpg",grayscale=False,target_size= (64,64))#loading of the image
x = img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
#pred = classifier.predict_classes(x)#predicting the classes
predict=model.predict(x)
classes_x=np.argmax(predict,axis=1)
classes_x

1/1 [==============================] - 0s 394ms/step


array([1])

In [19]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[classes_x[0]])
result

'Earthquake'